In [29]:
import cv2
import numpy as np
from deepface import DeepFace
import numpy as np
from scipy.spatial.distance import cosine, euclidean
from pathlib import Path
from deepface import DeepFace
import os

In [3]:
def embedding(image, model='ArcFace'):
    result = DeepFace.represent(img_path=image, model_name=model, enforce_detection=False)
    embedding = result[0]["embedding"]
    return np.array(embedding)

In [4]:
def preprocess_image(image_path, size=(160, 160)):
    img = cv2.imread(image_path)

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    enhanced = clahe.apply(gray)

    enhanced_bgr = cv2.cvtColor(enhanced, cv2.COLOR_GRAY2BGR)

    bright = cv2.convertScaleAbs(enhanced_bgr, alpha=1.1, beta=10)

    result = cv2.resize(bright, size, interpolation=cv2.INTER_LINEAR)

    return result

In [18]:
def cosine_similarity(emb1, emb2):
    if emb1 is None or emb2 is None:
        return -1.0
    emb1 = emb1 / np.linalg.norm(emb1)
    emb2 = emb2 / np.linalg.norm(emb2)
    return np.dot(emb1, emb2)


In [19]:
def find_matching_face(new_image_path, image_path, threshold=0.5):
    new_embedding = embedding(new_image_path, model='ArcFace')
    img_embedding = embedding(image_path, model='ArcFace')
    if new_embedding is None:
        return {
            "match_found": False,
            "message": f"Could not extract embedding from {new_image_path}",
            "closest_match": None,
            "similarity": -1.0
        }
    similarity = cosine_similarity(new_embedding, img_embedding)
    if similarity >= threshold:
        return {
            "match_found": True,
            "similarity": similarity
        }
    else:
        return {
            "match_found": False,
            "similarity": similarity
        }

In [ ]:
main_path = './test/download.png'
lst = []
result = []

for path in os.listdir('./downloaded_images'):
    if path.lower().endswith(('.jpg', '.jpeg', '.png')):
        full_path = os.path.join('./downloaded_images', path)
        x = find_matching_face(main_path, full_path)
        lst.append(x)


lst = lst.sort()
for i, l in enumerate(lst):
    if i == 3:
        break
    result.append(l)